In [135]:
from nltk.corpus import stopwords
#talvez seja necessário baixar as stopwords com o comando no CMD: python -m nltk.downloader stopwords
from string import punctuation
from nltk.tokenize import TweetTokenizer
import re

#______________________________________________________________________________________________________________________#

#Instância um objeto do tipo TweetTokenizer que já deixa tudo em minúsculo, trata espaços,  
# e elimina menções (mas não tira números).
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False) 

#definição das stopwords (inclui pontuação para eliminá-la, caso apareça)
stopwords = set(stopwords.words('portuguese') + list(punctuation))

def tokenizar_texto(texto_completo):
    texto_tokenizado_lst = tweet_tokenizer.tokenize(texto_completo)
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado_lst if palavra not in stopwords]
    return palavras_sem_stopwords

In [136]:
import nltk
import pandas as pd

# ler dados do arquivo seccionados a cada 200 tweets
dados = pd.read_json('tweets_222_23abr2020.json',convert_dates=False,lines=True,chunksize=2000)

In [137]:
# Criar Serie com os retweets dos 20000 primeiros tweets do arquivo
retweets = pd.Series(name = 'ID')

num = 0
for i in dados:
    num=num+1
    if(num<=50):
        retweets=retweets.append(i.dropna(subset=['retweeted_status'])['text'])
    else:
        break
retweets.size# 127121 retweets nos 200000 tweets # 12482 para 20000 # 62913 para 100000

C:\Users\vicda\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


62913

In [138]:
#criar Serie com a frequencia dos Bigramas nos tweets
freq_bi = {}#pd.Series(name = 'Palavras')
num_tweets_bi = {}#pd.Series(name = 'num_tweets')

#criar Serie com a frequencia dos unigramas nos tweets
freq_unigram = pd.Series(dtype=str,name = 'Palavras')
num_tweets_unigram = pd.Series(dtype=str,name = 'num_tweets')

add_uni = []# list para auxiliar na contagem de tweets em que aparece
add_bi = []# list para auxiliar na contagem de tweets em que aparece
for tweet in retweets.values:
    
    palavras = tokenizar_texto(tweet)
    palavras.remove('rt')
    bigramas_tweet = list(nltk.bigrams(palavras))
    add_uni.clear()
    add_bi.clear()
    
    for unigrama in palavras:
        if unigrama in freq_unigram:
            freq_unigram[unigrama] += 1
            if unigrama not in add_uni:
                num_tweets_unigram[unigrama] += 1
                add_uni.append(unigrama)
        else:
            freq_unigram[unigrama] = 1
            num_tweets_unigram[unigrama] = 1
            add_uni.append(unigrama)
            
    for bigrama in bigramas_tweet:
        if bigrama in freq_bi:
            freq_bi[bigrama] += 1
            if bigrama not in add_bi:
                num_tweets_bi[bigrama] += 1
                add_bi.append(bigrama)
        else:
            freq_bi[bigrama] = 1
            num_tweets_bi[bigrama] = 1
            add_bi.append(bigrama)
            
freq_bigram = pd.Series(freq_bi,name = 'Palavras')
num_tweets_bigram = pd.Series(num_tweets_bi,name = 'num_tweets')

In [139]:
# 50 unigramas que aparecem no maior numero de tweets nos 127121 retweets
num_tweets_unigram.sort_values(ascending = False)[:10]

…                          27722
quarentena                 26280
covid                       9617
19                          7112
cloroquina                  4568
pra                         4296
contra                      3503
brasil                      3405
corona                      3154
https://t.co/Bputjo9mS5     3140
Name: num_tweets, dtype: int64

In [140]:
# 50 bigramas que aparecem no maior numero de tweets nos 127121 retweets
num_tweets_bigram.sort_values(ascending = False)[:10]

covid       19                         7045
quarentena  https://t.co/Bputjo9mS5    3140
nessa       quarentena                 1948
testes      covid                      1502
vai         matar                      1150
”           diz                        1001
fome        vai                         973
covid       ”                           973
matar       covid                       973
bolsa       família                     972
Name: num_tweets, dtype: int64

In [141]:
# 50 unigramas mais ocorrentes nos tweets
freq_unigram.sort_values(ascending = False)[:10]

…                          27722
quarentena                 26858
covid                       9649
19                          7290
cloroquina                  4578
pra                         4476
contra                      3534
brasil                      3430
corona                      3158
https://t.co/Bputjo9mS5     3140
Name: Palavras, dtype: int64

In [142]:
# 50 bigramas mais ocorrentes nos tweets
freq_bigram.sort_values(ascending = False)[:10]

covid       19                         7072
quarentena  https://t.co/Bputjo9mS5    3140
nessa       quarentena                 1948
testes      covid                      1525
vai         matar                      1150
”           diz                        1001
covid       ”                           973
fome        vai                         973
matar       covid                       973
bolsa       família                     972
Name: Palavras, dtype: int64

In [134]:
freq_unigram.to_csv('num_ocorrencia_unigramas.csv',sep=';')
num_tweets_unigram.to_csv('num_tweets_unigrama_apararece.csv',sep=';')
freq_bigram.to_csv('num_ocorrencia_bigramas.csv',sep=';')
num_tweets_bigram.to_csv('num_tweets_bigrama_apararece.csv',sep=';')

PermissionError: [Errno 13] Permission denied: 'num_ocorrencia_unigramas.csv'